In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler



from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

# Load your dataset
df = pd.read_csv("data\data.csv")  # Make sure to replace "path/to/your/data.csv" with the actual path to your dataset

# If the labels are not yet encoded, you'll need to encode them
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Define features and labels
X = df.drop(['label', 'filename'], axis=1)
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



# Initialize models
log_clf = LogisticRegression(max_iter=5000, random_state=42)
rf_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(probability=True, random_state=42)
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
mlp_clf = MLPClassifier(random_state=42)
knn_clf = KNeighborsClassifier()

# Create the voting classifier ensemble
voting_clf = VotingClassifier(
    estimators=[
        ('lr', log_clf), 
        ('rf', rf_clf), 
        ('svc', svm_clf), 
        ('xgb', xgb_clf),
        ('mlp', mlp_clf),
        ('knn', knn_clf)
    ],
    voting='soft'
)

# Fit and predict as before
voting_clf.fit(X_train_scaled, y_train)
y_pred = voting_clf.predict(X_test_scaled)

# Evaluate the expanded model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.65
              precision    recall  f1-score   support

       blues       0.81      0.65      0.72        20
   classical       0.76      1.00      0.87        13
     country       0.61      0.63      0.62        27
       disco       0.65      0.52      0.58        21
      hiphop       0.56      0.60      0.58        15
        jazz       0.73      0.73      0.73        22
       metal       0.73      0.88      0.80        25
         pop       0.75      0.92      0.83        13
      reggae       0.57      0.52      0.55        23
        rock       0.29      0.24      0.26        21

    accuracy                           0.65       200
   macro avg       0.65      0.67      0.65       200
weighted avg       0.64      0.65      0.64       200

